In [26]:
# Dcard
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import json
import re
import time
import random
from pymongo import MongoClient
import pymongo
from datetime import datetime

In [27]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [28]:
def DcardCrawlar(tag,stop_page):
    allData = []
    page = 0
    count = 0
    while True:
        if page > int(stop_page):
            break
        try:
            url = "https://www.dcard.tw/_api/search/posts?field=topics&query="+str(tag)+"&offset="+str(page)+"&since=0"
            res = requests.get(url,headers=headers)
            if str(res) == "<Response [404]>":
                break

            rejs_list = json.loads(res.text)

            if rejs_list == []:
                break

            for i in rejs_list:
                ID = i['id']
                title = i['title']
                content = i['content']

                createdAt = i['createdAt']
                pattern = re.compile(r'\d\d\d\d-\d\d-\d\dT\d\d:\d\d:\d\d')
                date = re.search(pattern, createdAt).group()
                CreateTime = datetime.strptime(date,"%Y-%m-%dT%H:%M:%S")

                forumName = i['forumName']
                topics = i['topics']
                likeCount = i['likeCount']
                anonymousSchool = i['anonymousSchool']
                try:
                    school = i['school']
                except:
                    school = "匿名"

                page_r = 0
                Reply = []

                while True:
                    url_r = "https://www.dcard.tw/_api/posts/"+str(i['id'])+"/comments?after="+str(page_r)
                    res_r = requests.get(url_r,headers=headers)
                    if str(res_r) == "<Response [404]>":
                        break

                    rejs_r = json.loads(res_r.text)
                    if rejs_r == []:
                        break

                    time.sleep(random.randint(5,10))

                    for j in rejs_r:
                        try:
                            Name = j["school"]
                            Comment = j["content"]
                            Reply.append({"Name":Name,"Comment":Comment})
                        except:
                            pass
                    page_r += 30
                    
                ndf = pd.DataFrame(data = [{
                                            "Source":"Dcard",
                                            "Site":forumName,
                                            "Url":"https://www.dcard.tw/f/hkmacdaily/p/"+str(ID),
                                            "CreateTime":CreateTime,
                                            "Author":school,
                                            "Title":title,
                                            "Content":content,
                                            "Tag":topics,
                                            "Like":likeCount,
                                            "ReplyCount":len(Reply),
                                            "Reply":Reply}],
                                columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Reply','Tag','Like','ReplyCount'])
                
                count += 1
                allData.append(ndf)
        except:
            pass
        page += 30
        time.sleep(random.randint(5,10))
        if page > int(stop_page):
            break
    df = pd.concat(allData, ignore_index=True)
        
    return df

In [4]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_forum_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

In [5]:
# 政治_論壇
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.forum_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

In [32]:
data = "政治立場"
df = pd.DataFrame(DcardCrawlar(tag=data,stop_page=29))
dataframe_to_mongo(df)

In [2]:
data = "政治"
df = pd.DataFrame(DcardCrawlar(tag=data,stop_page=29))
dataframe_to_mongo(df)

In [29]:
data = "選舉"
df = pd.DataFrame(DcardCrawlar(tag=data,stop_page=29))
dataframe_to_mongo(df)

In [4]:
data = "立委"
df = pd.DataFrame(DcardCrawlar(tag=data,stop_page=29))
dataframe_to_mongo(df)

In [5]:
data = "2020"
df = pd.DataFrame(DcardCrawlar(tag=data,stop_page=29))
dataframe_to_mongo(df)